<a href="https://colab.research.google.com/github/usjunco/pangen/blob/master/infa_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 0.** If for some reason you encounter issues navigating the folders in the project or having errors, you can delete the project folder in the online colab environment by using the following code cell. If you are just starting to use this fresh colab notebook skip this step.

You can also go to the "Runtime" option and select "Disconnect and delete runtime".

In [ ]:
!cd ~
!pwd
!ls
!rm -r pangen/

/content
inf-test  sample_data


**Step 1.** Clone the github repository from https://github.com/usjunco/pangen.git , this will download the source code and also make the correct project structure to run the compression program.

In [1]:
!git clone https://github.com/usjunco/pangen
%cd pangen
!git checkout master #go to master branch to have the correct project structure
!ls

Cloning into 'pangen'...
remote: Enumerating objects: 567, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 567 (delta 101), reused 111 (delta 38), pack-reused 353 (from 1)
Receiving objects: 100% (567/567), 59.83 MiB | 7.39 MiB/s, done.
Resolving deltas: 100% (213/213), done.
Updating files: 100% (85/85), done.
/content/pangen
Already on 'master'
Your branch is up to date with 'origin/master'.
bkp   env     infa_main.ipynb  output		   README.md  sample_input  src
data  graphs  input	       output_compression  run_local  sequences     test


**Step 2.** Install the conda environment in /pangen/env/infa.yml to install dependencies needed to run the compression procedure.

If you would like to install the dependencies on your own, the important ones are:

* cd-hit 4.8.1  
* bowtie2 2.5.4
* cutadapt 3.5
* python 3.6.13
* perl 5.34.0
* pip 21.2.2

In [2]:
#install condacolab
!ls
!pip install -q condacolab
import condacolab
condacolab.install()
!ls
#create conda envionment in infa.yml file
!conda env create -f env/infa.yml --quiet
!conda env list

bkp   env     infa_main.ipynb  output		   README.md  sample_input  src
data  graphs  input	       output_compression  run_local  sequences     test
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...
bkp			env		 input		     README.md	   sequences
condacolab_install.log	graphs		 output		     run_local	   src
data			infa_main.ipynb  output_compression  sample_input  test
Channels:
 - defaults
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
# conda environments:
#
base                     /usr/local
infc                     /usr/local/envs/infc



**Step 3.** Run the following cell to create the function to upload your sequences into the compression procedure.

In [1]:
## Import file upload dependencies
from google.colab import files
import os
import shutil

def input():
  destination_dir = '/content/pangen/input/'
  os.makedirs(destination_dir, exist_ok=True)
  uploaded = files.upload()
  ## move the uploaded file to input directory
  for filename in uploaded.keys():
      destination = os.path.join(destination_dir, filename)
      shutil.move(filename, destination)
      filename_nf = destination.split('.')[0]
      shutil.move(destination, filename_nf)
  ## set and return input name
  name_list = filename_nf.split("/")
  input_fasta = name_list[-1]
  return input_fasta

**Step 4**. Upload your fasta file running the following cell and clicking the "Choose Files" button, this file should be in fasta format. (https://www.ncbi.nlm.nih.gov/genbank/fastaformat/) an example of this file can be found in /pangen/sample_input/testsample.fasta/

In [2]:
### this cell is to create the input directory and reciebe the fasta file
## import files dependences
input_fasta = input()

Saving testsample.fasta to testsample.fasta


**Step 5**. Upload your classes file, this file must be in a text base format, column wise separated by a "\t" caracter, where the first column is the "accession id" of each sequence in the input fasta file, the second column has to be the "class id" realted to the refered sequence. An example of this file is in /inf-test/sample_input/classes.dmp/

In [3]:
input_classes = input()

Saving classes.dmp to classes.dmp


**Step 6**. Upload your names file, this file has the names for all "class id" raws in the classes file. This file must be in a text file tab ("\t") separated, the first column being "class id", the second column being "class name". This format facilitates the making of the final pangenome file and makes it easier to use. An example of this file can be found in /inf-test/sample/release_examples/names.dmp/

In [4]:
input_names = input()

Saving names.dmp to names.dmp


In [5]:
## download the new merge.pl file
%cd /content/pangen/src/
!rm /content/pangen/src/merge.pl
!curl -O https://raw.githubusercontent.com/usjunco/pangen/master/src/merge.pl
print(input_names)

/content/pangen/src
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1052  100  1052    0     0   5037      0 --:--:-- --:--:-- --:--:--  5033
names


**Step 7.** Run the following cell to take the 3 input files and process them into the correct folders. This procedure will generate the necesary requisites to run the compression algorithm correctly.

In [6]:
## merge the data and put the files in the correct order
#  run the script with conda env
destination_dir = '/content/pangen/input/'
%cd /content/pangen/src/
ifld = destination_dir
dfld = "/content/pangen/data/"
os.makedirs(dfld, exist_ok=True)
sfld = "/content/pangen/sequences/"
os.makedirs(sfld, exist_ok=True)
!source activate infc && ls
!chmod +x merge.pl
#!source activate infc && ls {ifld}/{input_fasta}
!source activate infc && perl merge.pl {ifld}/{input_fasta} {ifld}/{input_classes} {ifld}/{input_names} {sfld}/{input_fasta}  {dfld}/sci_names.dmp {dfld}/acc_variants.list


/content/pangen/src
compressor  merge.pl


**Step 8.** Run the coompression algorithm with the follwing cell.
The results of the compression can be found in /pangen/output_compression/name_of_your_input/

///// you will need to change the names
///// of the final files ***italicized text***
"report.dmp" is the compression report, with general statistics about the compression procedure.

"your_file.pan" is the pangenome file, this file includes all dispensable and unique pieces for all classes defined in the classes file (second input).

Inside the "pangenome_unique" folder you can find separated fasta files with the ".u" extension. This files contains the unique pieces for each class that take part into the compression procedure.

**Make the headers descripcion of the pangenome file.**

In [7]:
#  run the script with conda env
%cd /content/pangen/src/compressor/
!source activate infc && ls
!chmod +x run_compress.sh
!source activate infc && ./run_compress.sh 0.94 0.958 200 50 100000 {input_fasta}


/content/pangen/src/compressor
cluster_acc_family.pl	    divider.pl		       lucas_compressor.sh
cluster_acc.pl		    enumerate.pl	       modify_headers.pl
cluster_acc_variant.pl	    enumerate.py	       remove.sh
cluster_element_counter.pl  enumerate.sh	       run_compress.sh
concat_clusters.sh	    kmer.pl		       uniques_concatenation.pl
concat_clusters_test.sh     label_clusters_variant.sh  uniques_entanglement.pl
counter.pl		    lucas_back		       uniques_gitrog.pl
count_lenghts_pieces.pl     lucas_compressor_b.sh      uniques_intanglement.pl
This is cutadapt 3.5 with Python 3.6.13
Command line parameters: --max-n 0 -o /content/pangen/src/compressor/../..//output_compression//testsample//kmers_clean.fasta /content/pangen/src/compressor/../..//output_compression//testsample//kmers_w.fasta
Processing reads on 1 core in single-end mode ...
Done           00:00:03       404,984 reads @   9.0 µs/read;   6.68 M reads/minute
Finished in 3.64 s (9 us/read; 6.68 M reads/minute).

=== Summary =